# A Basic Trend-Following System for Futures (Single Pass)

In this notebook we show a very simple strategy based on the crossing of two moving averages. The strategy can be used as starting point for developing more complex approaches. We use a fast single-pass implementation which however can lead to unintentional looking forward.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }
// disable widget scrolling

We work with complete time series and define allocation weights at once for all points in time. The computation is fast but, since the time series is processed in a single pass, forward looking can occur. Note that the performace of this system is not so good as the Sharpe ratio is negative. If you can write a system with a Sharpe ratio larger than 1, save your result and click on the **Submit** button in your **Development** area! Do not forget the final call to the **write** function as in this example.

In [ ]:
import xarray as xr

import qnt.ta as qnta
import qnt.stats as qnstats
import qnt.data as qndata
import qnt.output as qnout

# load data:
data = qndata.futures.load_data(min_date="2006-01-01")

# calc weights:
close = data.sel(field="close")
ma_slow = qnta.ema(close, periods=200, warm_periods=20)
ma_fast = qnta.ema(close, periods=50, warm_periods=10)
weights = xr.where(ma_fast > ma_slow, 1, -1)

# normalize if total exposure is larger>1 and fill missing values:
weights = qnout.clean(weights, data)

# calc stats (can be commented for submission):
stats = qnstats.calc_stat(data, weights.sel(time=slice("2006-01-01", None)))
display(stats.to_pandas().tail())

# check that weights are correct:
qnout.check(weights, data)

# write results, necessary for submission:
qnout.write(weights)


# Multi Pass analog


We use a multi-pass implementation which forbids forward looking and can be used for testing your idea.

We define a wrapper function for loading the data, and implement our strategy inside a function which returns allocation weights for a given point in time. Then we call the built-in backtesting function and perform the simulation avoiding potential looking-forward issues

```python
import xarray as xr

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata


def load_data(period):
    """Loads futures markets for the Classic Futures contest"""
    return qndata.futures.load_data(tail=period, dims=("time", "field", "asset"))


def strategy(data):
    close = data.sel(field="close")
    ma_slow = qnta.ema(close, periods=200, warm_periods=20).isel(time=-1)
    ma_fast = qnta.ema(close, periods=50, warm_periods=10).isel(time=-1)
    return xr.where(ma_fast > ma_slow, 1, -1)


weights = qnbt.backtest(
    # Classic Futures contest:
    competition_type="futures",
    load_data=load_data,
    lookback_period=365,
    start_date="2006-01-01",
    strategy=strategy,
    analyze=True,
    build_plots=True
)
```